# Contracts (Advanced)

This section covers advanced contract handling, particularly dealing with ambiguous contracts and using `reqContractDetails()` to get comprehensive contract information.

In [1]:
from ib_async import *
util.startLoop()

In [2]:
ib = IB()

In [3]:
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

## Case 1: Unambiguous (one contract exists)

When you provide specific details, Interactive Brokers can uniquely identify the contract:

In [4]:
contract = Stock("AAPL", "SMART", "USD")
contract

Stock(symbol='AAPL', exchange='SMART', currency='USD')

In [5]:
cds = ib.reqContractDetails(contract)
cds

[ContractDetails(contract=Contract(secType='STK', conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), marketName='NMS', minTick=0.01, orderTypes='ACTIVETIM,AD,ADDONT,ADJUST,ALERT,ALGO,ALLOC,AON,AVGCOST,BASKET,BENCHPX,CASHQTY,COND,CONDORDER,DARKONLY,DARKPOLL,DAY,DEACT,DEACTDIS,DEACTEOD,DIS,DUR,GAT,GTC,GTD,GTT,HID,IBKRATS,ICE,IMB,IOC,LIT,LMT,LOC,MIDPX,MIT,MKT,MOC,MTL,NGCOMB,NODARK,NONALGO,OCA,OPG,OPGREROUT,PEGBENCH,PEGMID,POSTATS,POSTONLY,PREOPGRTH,PRICECHK,REL,REL2MID,RELPCTOFS,RPI,RTH,SCALE,SCALEODD,SCALERST,SIZECHK,SMARTSTG,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,SWEEP,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF', validExchanges='SMART,AMEX,NYSE,CBOE,PHLX,ISE,CHX,ARCA,NASDAQ,DRCTEDGE,BEX,BATS,EDGEA,BYX,IEX,EDGX,FOXRIVER,PEARL,NYSENAT,LTSE,MEMX,IBEOS,OVERNIGHT,TPLUS0,PSX,T24X', priceMagnifier=1, underConId=0, longName='APPLE INC', contractMonth='', industry='Technology', category='Computers', subcategory='Computers', 

In [6]:
len(cds)

1

In [7]:
cds[0].contract

Contract(secType='STK', conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS')

In [8]:
cds[0].contract.conId

265598

The `reqContractDetails()` function returns a list of `ContractDetails` objects. For an unambiguous contract like AAPL with specific exchange and currency, we get exactly one result.

## Case 2: Ambiguous (multiple contracts match)

When you provide incomplete information, IB may find multiple matching contracts:

In [9]:
contract = Stock("AAPL")  # No exchange or currency specified
cds = ib.reqContractDetails(contract)
len(cds)

31

In [10]:
# Show the first few contracts found
for i in range(min(5, len(cds))):
    contract_detail = cds[i]
    contract = contract_detail.contract
    print(f"{i+1}. {contract.symbol} - {contract.currency} - {contract.primaryExchange} - ConId: {contract.conId}")

1. AAPL - USD - NASDAQ - ConId: 265598
2. AAPL - CHF - EBS - ConId: 273982664
3. AAPL - CAD - TSE - ConId: 532640894
4. AAPL - USD - NASDAQ - ConId: 265598
5. AAPL - USD - NASDAQ - ConId: 265598


When searching for just "AAPL" without specifying exchange or currency, IB finds multiple matches across different exchanges and currencies (USD, CHF, CAD, MXN, etc.).

## Accessing Contract Details

You can access the contract from each `ContractDetails` object:

In [11]:
# Access the first contract from the search results
cds[0].contract

Contract(secType='STK', conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS')

In [12]:
# Get the contract ID (conId) for the first contract
cds[0].contract.conId

265598

## Exploring Contract Details Properties

The `ContractDetails` object contains much more information than just the contract. You can access various properties:

In [13]:
# Access order types available for this contract
cds[0].orderTypes

'ACTIVETIM,AD,ADDONT,ADJUST,ALERT,ALGO,ALLOC,AON,AVGCOST,BASKET,BENCHPX,CASHQTY,COND,CONDORDER,DARKONLY,DARKPOLL,DAY,DEACT,DEACTDIS,DEACTEOD,DIS,DUR,GAT,GTC,GTD,GTT,HID,IBKRATS,ICE,IMB,IOC,LIT,LMT,LOC,MIDPX,MIT,MKT,MOC,MTL,NGCOMB,NODARK,NONALGO,OCA,OPG,OPGREROUT,PEGBENCH,PEGMID,POSTATS,POSTONLY,PREOPGRTH,PRICECHK,REL,REL2MID,RELPCTOFS,RPI,RTH,SCALE,SCALEODD,SCALERST,SIZECHK,SMARTSTG,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,SWEEP,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF'

The `orderTypes` property shows all the order types that are available for this specific contract (e.g., 'MKT', 'LMT', 'STP', etc.).

In [14]:
# Other useful properties you can access from ContractDetails:
print("Market name:", cds[0].marketName)
print("Minimum tick:", cds[0].minTick)
print("Price magnifier:", cds[0].priceMagnifier)
print("Valid exchanges:", cds[0].validExchanges)
print("Time zone ID:", cds[0].timeZoneId)

Market name: NMS
Minimum tick: 0.01
Price magnifier: 1
Valid exchanges: SMART,AMEX,NYSE,CBOE,PHLX,ISE,CHX,ARCA,NASDAQ,DRCTEDGE,BEX,BATS,EDGEA,BYX,IEX,EDGX,FOXRIVER,PEARL,NYSENAT,LTSE,MEMX,IBEOS,OVERNIGHT,TPLUS0,PSX,T24X
Time zone ID: US/Eastern


## Converting Contracts to DataFrame

You can convert contract details to a pandas DataFrame for easier analysis and visualization:

In [15]:
# Convert contract details to DataFrame using util.df() shortcut
contracts_df = util.df(cds)  # converting to pd DataFrame shortcut (alternative: pd.DataFrame())
contracts_df

,contract,marketName,minTick,orderTypes,validExchanges,priceMagnifier,underConId,longName,contractMonth,industry,...,callable,putable,coupon,convertible,maturity,issueDate,nextOptionDate,nextOptionType,nextOptionPartial,notes
0,"Contract(secType='STK', conId=265598, symbol='...",NMS,0.010,"ACTIVETIM,AD,ADDONT,ADJUST,ALERT,ALGO,ALLOC,AO...","SMART,AMEX,NYSE,CBOE,PHLX,ISE,CHX,ARCA,NASDAQ,...",1,0,APPLE INC,,Technology,...,False,False,0,False,,,,,False,
1,"Contract(secType='STK', conId=273982664, symbo...",AAPL,0.001,"ACTIVETIM,AD,ADJUST,ALERT,ALGO,ALGOLTH,ALLOC,A...","SMART,EBS",1,0,APPLE INC,,Technology,...,False,False,0,False,,,,,False,
2,"Contract(secType='STK', conId=532640894, symbo...",AAPL,0.005,"ACTIVETIM,AD,ADJUST,ALERT,ALGO,ALLOC,AUC,AVGCO...","SMART,TSE",1,0,APPLE INC-CDR,,Technology,...,False,False,0,False,,,,,False,
3,"Contract(secType='STK', conId=265598, symbol='...",NMS,0.010,"ACTIVETIM,AD,ADJUST,ALERT,ALGOOPG,ALLOC,AVGCOS...","SMART,AMEX,NYSE,CBOE,PHLX,ISE,CHX,ARCA,NASDAQ,...",1,0,APPLE INC,,Technology,...,False,False,0,False,,,,,False,
4,"Contract(secType='STK', conId=265598, symbol='...",NMS,0.010,"ACTIVETIM,AD,ADJUST,ALERT,ALGOOPG,ALLOC,AVGCOS...","SMART,AMEX,NYSE,CBOE,PHLX,ISE,CHX,ARCA,NASDAQ,...",1,0,APPLE INC,,Technology,...,False,False,0,False,,,,,False,
5,"Contract(secType='STK', conId=265598, symbol='...",NMS,0.010,"ACTIVETIM,AD,ADJUST,ALERT,ALLOC,AVGCOST,BASKET...","SMART,AMEX,NYSE,CBOE,PHLX,ISE,CHX,ARCA,NASDAQ,...",1,0,APPLE INC,,Technology,...,False,False,0,False,,,,,False,
6,"Contract(secType='STK', conId=265598, symbol='...",NMS,0.010,"ACTIVETIM,AD,ADJUST,ALERT,ALLOC,AVGCOST,BASKET...","SMART,AMEX,NYSE,CBOE,PHLX,ISE,CHX,ARCA,NASDAQ,...",1,0,APPLE INC,,Technology,...,False,False,0,False,,,,,False,
7,"Contract(secType='STK', conId=265598, symbol='...",NMS,0.010,"ACTIVETIM,AD,ADJUST,ALERT,ALLOC,AVGCOST,BASKET...","SMART,AMEX,NYSE,CBOE,PHLX,ISE,CHX,ARCA,NASDAQ,...",1,0,APPLE INC,,Technology,...,False,False,0,False,,,,,False,
8,"Contract(secType='STK', conId=265598, symbol='...",NMS,0.010,"ACTIVETIM,AD,ADJUST,ALERT,ALLOC,AVGCOST,BASKET...","SMART,AMEX,NYSE,CBOE,PHLX,ISE,CHX,ARCA,NASDAQ,...",1,0,APPLE INC,,Technology,...,False,False,0,False,,,,,False,
9,"Contract(secType='STK', conId=265598, symbol='...",NMS,0.010,"ACTIVETIM,AD,ADJUST,ALERT,ALGOOPG,ALLOC,AVGCOS...","SMART,AMEX,NYSE,CBOE,PHLX,ISE,CHX,ARCA,NASDAQ,...",1,0,APPLE INC,,Technology,...,False,False,0,False,,,,,False,


In [16]:
# You can also extract just the contracts and convert them to DataFrame
contracts_only = [cd.contract for cd in cds]
contracts_only_df = util.df(contracts_only)
contracts_only_df

,secType,conId,symbol,lastTradeDateOrContractMonth,strike,right,multiplier,exchange,primaryExchange,currency,localSymbol,tradingClass,includeExpired,secIdType,secId,description,issuerId,comboLegsDescrip,comboLegs,deltaNeutralContract
0,STK,265598,AAPL,,0.0,,,SMART,NASDAQ,USD,AAPL,NMS,False,,,,,,[],None
1,STK,273982664,AAPL,,0.0,,,SMART,EBS,CHF,AAPL,AAPL,False,,,,,,[],None
2,STK,532640894,AAPL,,0.0,,,SMART,TSE,CAD,AAPL,AAPL,False,,,,,,[],None
3,STK,265598,AAPL,,0.0,,,AMEX,NASDAQ,USD,AAPL,NMS,False,,,,,,[],None
4,STK,265598,AAPL,,0.0,,,NYSE,NASDAQ,USD,AAPL,NMS,False,,,,,,[],None
5,STK,265598,AAPL,,0.0,,,CBOE,NASDAQ,USD,AAPL,NMS,False,,,,,,[],None
6,STK,265598,AAPL,,0.0,,,PHLX,NASDAQ,USD,AAPL,NMS,False,,,,,,[],None
7,STK,265598,AAPL,,0.0,,,ISE,NASDAQ,USD,AAPL,NMS,False,,,,,,[],None
8,STK,265598,AAPL,,0.0,,,CHX,NASDAQ,USD,AAPL,NMS,False,,,,,,[],None
9,STK,265598,AAPL,,0.0,,,ARCA,NASDAQ,USD,AAPL,NMS,False,,,,,,[],None


The `util.df()` function is a convenient shortcut provided by ib_async to convert lists of objects to pandas DataFrames. This is particularly useful when you have multiple contracts and want to compare them in a tabular format.

## Using Contract Details vs Qualify Contracts

Both `reqContractDetails()` and `qualifyContracts()` can be used to resolve contract information, but they serve different purposes:

- **`reqContractDetails()`**: Returns comprehensive contract details and can handle ambiguous searches
- **`qualifyContracts()`**: Validates contracts and fills missing fields, but throws an error for ambiguous matches

Use `reqContractDetails()` when you want to explore all possible matches for a symbol.

In [17]:
# Clean up - disconnect when done
ib.disconnect()

'Disconnecting from 127.0.0.1:7497, 205 B sent in 10 messages, 49.0 kB received in 485 messages, session time 73.2 s.'